In [1]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import GPT4All

In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
pip install gpt4all
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [19]:
def sendTheMessage(driver,wait, sendingText):
    message_box_path = '//*[@id="main"]/footer/div[1]/div/span[2]/div/div[2]/div[1]/div/div[2]'
    message_box = wait.until(EC.element_to_be_clickable((By.XPATH, message_box_path)))
    ActionChains(driver).send_keys(sendingText).send_keys(Keys.RETURN).perform()
    

def clickTheTarget(driver,wait, messages):
    target = 'Shreyas IND' 
    contact_path = f'//span[contains(@title,"{target}")]'
    contact = wait.until(EC.element_to_be_clickable((By.XPATH, contact_path)))
    contact.click()

    for sendingText in messages:
        sendTheMessage(driver,wait, sendingText)

In [22]:
driver = webdriver.Chrome()
driver.get("https://web.whatsapp.com/")
wait = WebDriverWait(driver, 100)
target = 'Deep learning '
contact_path = f'//span[contains(@title,"{target}")]'
contact = wait.until(EC.element_to_be_clickable((By.XPATH, contact_path)))
contact.click()

num_messages_to_retrieve = 6

for _ in range(num_messages_to_retrieve // 10):
    driver.execute_script("window.scrollBy(0, -800);")
    time.sleep(2)

messages = []
message_elements = driver.find_elements(By.XPATH, '//div[@class="copyable-text"][@data-pre-plain-text]')
for message_element in message_elements[-num_messages_to_retrieve:]:
    message = message_element.text
    messages.append(message)
print(messages)
clickTheTarget(driver,wait, messages)

time.sleep(10)
driver.quit()

['Trichy (vaiampatti) to Dindukkal 20 feet closed truck required 2 trip available please call me at 9363305939', 'Panipat to alleppy 32 f m axil and 32 f s axil truck required call me at 9363305939', 'Thirunelveli to Vaizak Ton 1 Insulated truck required Dosth badadosth truck require 9363305939', 'Tirupur in banglore 20/22 feet open truck required call me at 9363305939', 'Mailam near pondicherry 2 Chennai 17 feet open truck required call me at 9363305939', 'Chennai to trichi 17 f open truck required call me at 9363305939']


In [23]:
len(messages)

6

In [24]:
template = """Find source , destination , size of the truck and mobile number from the given data and return it as 1D json format without giving any other extra members for each of them. Just source, destination, mobile number (in 10 digits) and size : {text}
              Just the JSON format (No other samples or information) :"""
prompt = PromptTemplate(template=template, input_variables=["text"])

In [25]:
local_path = (
    "./gpt4all-falcon-newbpe-q4_0.gguf"
)

callbacks = [StreamingStdOutCallbackHandler()]

llm = GPT4All(model=local_path, callbacks=callbacks, verbose=True)

llm = GPT4All(model=local_path, backend="gptj", callbacks=callbacks, verbose=True)

In [26]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [27]:

for text in messages:
    print(text)
    print(llm_chain.run(text))
    print()

Trichy (vaiampatti) to Dindukkal 20 feet closed truck required 2 trip available please call me at 9363305939

{
"source": "Trichy",
"destination": "Dindukkal",
"size": "20 feet",
"mobile number": "93633059",
"trip required": 2,
"available": true,
"call me at": "93633059"
}
{
"source": "Trichy",
"destination": "Dindukkal",
"size": "20 feet",
"mobile number": "93633059",
"trip required": 2,
"available": true,
"call me at": "93633059"
}

Panipat to alleppy 32 f m axil and 32 f s axil truck required call me at 9363305939

{
"source": "Panipat",
"destination": "Alleppy",
"size": "32 f m axil and 32 f s axil",
"mobile number": "9363305939"
}
{
"source": "Panipat",
"destination": "Alleppy",
"size": "32 f m axil and 32 f s axil",
"mobile number": "9363305939"
}

Thirunelveli to Vaizak Ton 1 Insulated truck required Dosth badadosth truck require 9363305939

{
"source": "Thirunelveli",
"destination": "Vaizak Ton 1",
"size": "Insulated",
"mobile_number": "93633059",
"dosth_badadosth_truck_require